In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [ ]:
!pip install konlpy
!pip install img2vec_pytorch

     |████████████████████████████████| 19.4 MB 367 kB/s 
     |████████████████████████████████| 448 kB 95.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch
import joblib
import json
from tqdm.notebook import tqdm
from preprocess import preprocessing, text2vector, image2vector

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# 예측에 사용할 feature 이름 
with open('data/feature_dict.json', 'r') as f:
    feature_dict = json.load(f)
categorical, continuous, candidates = feature_dict['categorical'], feature_dict['continuous'], feature_dict['candidates']
features = categorical + continuous

# Label Encoder
with open('data/cat_1_dict.json', 'r') as f:
    cat_label = json.load(f)
with open('data/place_dict.json', 'r') as f:
    place_label = json.load(f)
# Standard Scaler
scaler = joblib.load('model/robust_scaler.pkl')
# 학습된 DeepFM 모델
model_dfm = torch.load('model/deepfm_final.h5')

# CTR 예측

### 가상 유저군 생성

In [ ]:
from itertools import product

# 유저 정보 (median 사용)
users = {'bid_price': [50],
         'viewer_gender': [1,2],
         'viewer_age': range(10, 80),
         'viewer_chat_count': [0.0],
        'viewer_following_count': [1.0],
        'viewer_parcel_post_count': [0.0],
        'viewer_pay_count': [0.0],
        'viewer_transfer_count': [0.0]
         }

# 유저 생성
user_list = []
for i, values in enumerate(list(product(*users.values()))):
    user = dict(zip(users.keys(), values))
    user_list.append(user)

### 각 소분류별로 CTR 예측

In [ ]:
# ----------------------------------------------------------------
# cat_id_list = [310100, 310180, 310250, 310080, 310190, 310200]
# ----------------------------------------------------------------
# cat_id_list = [310220, 320080, 320100, 320200, 320190, 320210]
# ----------------------------------------------------------------
# cat_id_list = [310090, 310110, 310240, 310230]
# ----------------------------------------------------------------
# cat_id_list = [310260, 310040, 310050, 310150, 310140, 310160, 310120, 310070, 310060]
# ----------------------------------------------------------------
# cat_id_list = [320040, 320060, 320050, 320120, 320110]
# ----------------------------------------------------------------
cat_id_list = [320130, 320090, 320140, 320070, 320150, 320160, 320180]

for cat_id in tqdm(cat_id_list):
    # 데이터 불러오기 (소분류별로 하나씩)
    df = pd.read_csv(f'data/new_crawling/new_crawling_{cat_id}.csv')
    # 데이터 전처리
    df = preprocessing(df)
    print('-'*45)
    print(f'대분류: {df.content_cat_1[0]}, 중분류: {df.content_cat_2[0]}, 소분류: {df.content_cat_3[0]}')
    print('-'*45)
    # 텍스트 전처리 및 벡터화
    text2vec = text2vector(df)
    # 텍스트 벡터 저장 (cid : vector 형식)
    with open(f'data/의류/text/text_vec_{cat_id}.json', 'w') as f:
        json.dump(text2vec, f)
    # 이미지 벡터화
    img2vec = image2vector(df)
    # 이미지 벡터 저장 (cid : vector 형식)
    with open(f'data/의류/image/image_vec_{cat_id}.json', 'w') as f:
        json.dump({key:[float(v) for v in value] for key,value in img2vec.items()}, f)    

    # DeepFM 모델 input 형식에 맞게 (이미지, 텍스트)
    predict_input = {'image_vec': np.array([img2vec[cid] for cid in df['content_id']]),
                    'text_vec': np.array([text2vec[cid] for cid in df['content_id']])}
    
    # 위에서 생성한 유저 140명에 대해 CTR 예측
    print('CTR 예측 시작')

    # CTR 저장할 df 생성
    result_df = pd.DataFrame()
    for demo_user in tqdm(user_list):
        # 각 유저에 대한 ctr 저장할 df 생성
        user_ctr_df = pd.DataFrame({'content_id': df.content_id, 
                                    'gender': demo_user['viewer_gender'],
                                    'age': demo_user['viewer_age'],
                                    })
        
        # 예측 input으로 사용할 데이터만 뽑기
        candidates_df = df[candidates].copy()        
        
        # 각 유저마다 특성을 입력(나이, 성별 등)
        for feat in demo_user.keys():
            candidates_df[feat] = demo_user[feat]
        # 학습했을때와 같은 순서로 컬럼 정렬
        candidates_df = candidates_df[features]

        # Label Encoding
        candidates_df['content_cat_1'] = candidates_df['content_cat_1'].apply(lambda x: cat_label[x])
        # Standard Scaling
        candidates_df[continuous] = scaler.transform(candidates_df[continuous])
        
        # DeepFM input 형태에 맞게(정형 데이터)
        candidates_input = {name: candidates_df[name] for name in features}
        # 정형 + 비정형 input 합치기
        predict_input.update(candidates_input)

        # CTR 예측
        CTR_dfm = model_dfm.predict(predict_input, 512)

        # 예측한 CTR 저장
        user_ctr_df['ctr'] = CTR_dfm
        result_df = result_df.append(user_ctr_df)
    
    # 컨텐츠, CTR 정보 따로 저장
    df.to_csv(f'data/의류/content_{cat_id}.csv', index=False)
    result_df.to_csv(f'data/의류/ctr_{cat_id}.csv', index=False)

  0%|          | 0/7 [00:00<?, ?it/s]

---------------------------------------------
대분류: 남성의류, 중분류: 반바지, 소분류: 기타(반바지)
---------------------------------------------
텍스트 전처리 및 벡터화


  0%|          | 0/1884 [00:00<?, ?it/s]

이미지 불러오기 및 벡터화


  0%|          | 0/1884 [00:00<?, ?it/s]

...

  0%|          | 0/1994 [00:00<?, ?it/s]

이미지 불러오기 및 벡터화


  0%|          | 0/1994 [00:00<?, ?it/s]

CTR 예측 시작


  0%|          | 0/140 [00:00<?, ?it/s]

# NEW

In [ ]:
import os

In [ ]:
file_list = [filename for filename in os.listdir('data/의류') if ('content' in filename)&('(' not in filename)]

for file_name in tqdm(file_list):
    cat_id = file_name[8:14]
    # 데이터 불러오기 (소분류별로 하나씩)
    df = pd.read_csv('data/의류/' + file_name)
    # 데이터 전처리
    # df = preprocessing(df)
    print('-'*45)
    print(f'대분류: {df.content_cat_1[0]}, 중분류: {df.content_cat_2[0]}, 소분류: {df.content_cat_3[0]}')
    print('-'*45)

    with open(f'data/의류/text/text_vec_{cat_id}.json', 'r') as f:
        text2vec = json.load(f)
    # 이미지 벡터 저장 (cid : vector 형식)
    with open(f'data/의류/image/image_vec_{cat_id}.json', 'r') as f:
        img2vec = json.load(f)

    # DeepFM 모델 input 형식에 맞게 (이미지, 텍스트)
    predict_input = {'image_vec': np.array([img2vec[str(cid)] for cid in df['content_id']]),
                    'text_vec': np.array([text2vec[str(cid)] for cid in df['content_id']])}
    
    # 위에서 생성한 유저 140명에 대해 CTR 예측
    print('CTR 예측 시작')

    # CTR 저장할 df 생성
    result_df = pd.DataFrame()
    for demo_user in tqdm(user_list):
        # 각 유저에 대한 ctr 저장할 df 생성
        user_ctr_df = pd.DataFrame({'content_id': df.content_id, 
                                    'gender': demo_user['viewer_gender'],
                                    'age': demo_user['viewer_age'],
                                    })
        
        # 예측 input으로 사용할 데이터만 뽑기
        candidates_df = df[candidates].copy()        
        
        # 각 유저마다 특성을 입력(나이, 성별 등)
        for feat in demo_user.keys():
            candidates_df[feat] = demo_user[feat]
        # 학습했을때와 같은 순서로 컬럼 정렬
        candidates_df = candidates_df[features]

        # Label Encoding
        candidates_df['content_cat_1'] = candidates_df['content_cat_1'].apply(lambda x: cat_label[x])
        # Standard Scaling
        candidates_df[continuous] = scaler.transform(candidates_df[continuous])
        
        # DeepFM input 형태에 맞게(정형 데이터)
        candidates_input = {name: candidates_df[name] for name in features}
        # 정형 + 비정형 input 합치기
        predict_input.update(candidates_input)

        # CTR 예측
        CTR_dfm = model_dfm.predict(predict_input, 512)

        # 예측한 CTR 저장
        user_ctr_df['ctr'] = CTR_dfm
        result_df = result_df.append(user_ctr_df)
    
    # 컨텐츠, CTR 정보 따로 저장
    # df.to_csv(f'data/의류/content_{cat_id}_new.csv', index=False)
    result_df.to_csv(f'data/의류/ctr_{cat_id}_new.csv', index=False)

  0%|          | 0/36 [00:00<?, ?it/s]

---------------------------------------------
대분류: 여성의류, 중분류: 자켓, 소분류: 바람막이
---------------------------------------------
CTR 예측 시작


  0%|          | 0/140 [00:00<?, ?it/s]

---------------------------------------------
대분류: 여성의류, 중분류: 정장, 소분류: 자켓
---------------------------------------------
CTR 예측 시작


  0%|          | 0/140 [00:00<?, ?it/s]

...

  0%|          | 0/140 [00:00<?, ?it/s]

---------------------------------------------
대분류: 남성의류, 중분류: 테마/이벤트, 소분류: 테마/이벤트
---------------------------------------------
CTR 예측 시작


  0%|          | 0/140 [00:00<?, ?it/s]

---------------------------------------------
대분류: 여성의류, 중분류: 원피스, 소분류: 롱 원피스
---------------------------------------------
CTR 예측 시작


  0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
result_df

,content_id,gender,age,ctr
0,175785616,1,10,0.000303
1,120118318,1,10,0.004735
2,171055201,1,10,0.003712
3,171421179,1,10,0.012558
4,122754723,1,10,0.004395
...,...,...,...,...
1920,157019585,2,79,0.005983
1921,174793047,2,79,0.013453
1922,175836854,2,79,0.007652
1923,172639475,2,79,0.004660
